In [1]:
# Importing libraries and establishing connection to SQL database

import numpy as np
import pandas as pd
import sqlite3 as sql
conn = sql.connect('../zippedData/im.db')

In [2]:
# Declaration of base dataframes

# Studios and gross revenue - 3387 rows
# https://www.boxofficemojo.com/
gross_df = pd.read_csv('../zippedData/bom.movie_gross.csv')
# The movie database - https://www.themoviedb.org/?language=en-US
# Reviews - 26517 rows, release date, popularity score, votes?
reviews_df = pd.read_csv('../zippedData/tmdb.movies.csv')
# Budget and gross revenues - 5782 rows
# https://www.the-numbers.com/
budget_df = pd.read_csv('../zippedData/tn.movie_budgets.csv')
# Movie ratings (R, etc.), genre, director, writer, theater/dvd date, length, box office amount, studio - 1560 rows
movie_info_df = pd.read_csv('../zippedData/rt.movie_info.tsv', sep='\t')
# Rotten tomatoes - reviews, crtiic, review rating, critic quality - 54,432 rows
rt_reviews_df = pd.read_csv('../zippedData/rt.reviews.tsv', sep='\t', encoding='latin-1')
# Ratings and genres by director from imdb database
director_and_ratings_df = pd.read_sql(
"""SELECT DISTINCT b.*,
            r.averagerating AS avg_rating,
            r.numvotes AS num_votes,
            p.primary_name AS director,
            p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN directors d
ON b.movie_id = d.movie_id
LEFT JOIN persons p
ON d.person_id = p.person_id
""", conn)
# Ratings and genres by writers from imdb database
writers_df = pd.read_sql("""
SELECT DISTINCT b.*, r.averagerating AS avg_rating, r.numvotes AS num_votes, p.primary_name AS 'writer', p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN writers w
ON b.movie_id = w.movie_id
LEFT JOIN persons p
ON w.person_id = p.person_id

""", conn)

In [3]:
# Reformatting data in budget_df

# Turned all the monetary strings into ints
budget_df[budget_df.columns[3:]] = budget_df[budget_df.columns[3:]].replace('[\$\.]|,', '', regex=True).astype('float')
# Added column of world wide gross - production budget to get the net profit
budget_df['profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']

In [4]:
# Merging budget dataframe and gross dataframe by domestic gross
# Merged on movie from budget_df and title from gross_df
merged_budget_and_gross_df = pd.merge(budget_df, gross_df, how = "outer", left_on = 'movie', right_on = 'title')
# Dropping id and year columns, since year is represented in release_date (may need to go back, since some release_dates are missing)
merged_budget_and_gross_df.drop(['id', 'year'], axis=1, inplace=True)
# Merging gross_y and gross_x columns, preferring gross_x which seem to have higher precision
merged_budget_and_gross_df.domestic_gross_x = merged_budget_and_gross_df.domestic_gross_x.fillna(merged_budget_and_gross_df.domestic_gross_y)
# Merging movie and title, preferring movie
merged_budget_and_gross_df.movie = merged_budget_and_gross_df.movie.fillna(merged_budget_and_gross_df.title)
# Dropping redundant gross and title columns
merged_budget_and_gross_df.drop(['domestic_gross_y', 'title'], axis=1, inplace=True)
# Top 25 movies by profit
merged_budget_and_gross_df.sort_values('profit', ascending=False).head(25)

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,profit,studio,foreign_gross
0,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,NaN,NaN
46,"Dec 19, 1997",Titanic,200000000.0,659363944.0,2.208208e+09,2.008208e+09,NaN,NaN
6,"Apr 27, 2018",Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,1.748134e+09,BV,"1,369.5"
5,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,1.747311e+09,NaN,NaN
34,"Jun 12, 2015",Jurassic World,215000000.0,652270625.0,1.648855e+09,1.433855e+09,Uni.,"1,019.4"
72,"Apr 3, 2015",Furious 7,190000000.0,353007020.0,1.518723e+09,1.328723e+09,Uni.,"1,163.0"
26,"May 4, 2012",The Avengers,225000000.0,623279547.0,1.517936e+09,1.292936e+09,NaN,NaN
274,"Jul 15, 2011",Harry Potter and the Deathly Hallows: Part II,125000000.0,381193157.0,1.341693e+09,1.216693e+09,NaN,NaN
45,"Feb 16, 2018",Black Panther,200000000.0,700059566.0,1.348258e+09,1.148258e+09,BV,646900000
121,"Jun 22, 2018",Jurassic World: Fallen Kingdom,170000000.0,417719760.0,1.305773e+09,1.135773e+09,Uni.,891800000


In [5]:
# Top 25 Directors by average rating and means for runtimes and number of votes
director_and_ratings_df[['genres','runtime_minutes', 'avg_rating','num_votes']].groupby('genres').mean().sort_values(by='avg_rating', ascending = False).head(25)

,runtime_minutes,avg_rating,num_votes
genres,,,
"Comedy,Documentary,Fantasy",70.00,9.400000,5.000000
"Documentary,Family,Musical",59.00,9.300000,19.000000
"History,Sport",NaN,9.200000,5.000000
Game-Show,130.00,9.000000,7.000000
"Music,Mystery",NaN,9.000000,5.000000
"Drama,Fantasy,War",NaN,8.800000,22.000000
"Documentary,News,Reality-TV",61.00,8.800000,8.000000
"Documentary,News,Sport",85.00,8.800000,25.000000
"Comedy,Drama,Reality-TV",NaN,8.800000,15.000000


In [6]:
# Top 25 Directors by average rating and means for runtimes and number of votes
director_and_ratings_df[['director','runtime_minutes', 'avg_rating','num_votes']].groupby('director').mean().sort_values(by='avg_rating', ascending = False).head(25)
# We may want to include some sort of statistic that combines avg_rating and num_votes somehow
# A director with high avg_rating but low num_votes is less impressive than one with slightly less avg_rating but high num_votes

,runtime_minutes,avg_rating,num_votes
director,,,
Loreto Di Cesare,70.0,10.0,8.0
Lindsay Thompson,65.0,10.0,7.0
Masahiro Hayakawa,59.0,10.0,5.0
Chad Carpenter,NaN,10.0,5.0
Michiel Brongers,48.0,10.0,5.0
Emre Oran,77.0,10.0,6.0
Stephen Peek,NaN,10.0,20.0
Tristan David Luciotti,NaN,10.0,6.0
Ivana Diniz,52.0,10.0,5.0


In [7]:
# Top 25 Writers by average rating and means for runtimes and number of votes
writers_df[['writer','runtime_minutes', 'avg_rating','num_votes']].groupby('writer').mean().sort_values(by='avg_rating', ascending = False).head(25)

,runtime_minutes,avg_rating,num_votes
writer,,,
Emre Oran,77.0,10.0,6.0
Ana Beatriz Nogueira,52.0,10.0,5.0
Brian Baucum,70.0,10.0,8.0
Ivana Diniz,52.0,10.0,5.0
Lindsay Thompson,65.0,10.0,7.0
Cristina Duarte,93.0,10.0,5.0
Heather Augustyn,99.0,10.0,5.0
Loreto Di Cesare,70.0,10.0,8.0
Daniel Alexander,100.0,10.0,5.0


In [8]:
gross_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [9]:
reviews_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [10]:
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.000000e+00,-7.000000e+03
5778,79,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05,2.344950e+05
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03


In [11]:
movie_info_df.rename(columns = {"genre" : "genres"}, inplace = True)
movie_info_df

,id,synopsis,rating,genres,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [14]:
movie_info_df["genres"] = movie_info_df["genres"].str.split(pat = "|")
movie_info_df = movie_info_df.explode("genres")
movie_info_df

,id,synopsis,rating,genres,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
0,1,"This gritty, fast-paced, and innovative police...",R,Classics,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
0,1,"This gritty, fast-paced, and innovative police...",R,Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
...,...,...,...,...,...,...,...,...,...,...,...,...
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Action and Adventure,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Art House and International,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Comedy,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Drama,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures


In [15]:
rt_reviews_df

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [16]:
director_and_ratings_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,director,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Mani Kaul,"director,writer,actor"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Orson Welles,"actor,director,writer"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,Hrishikesh Mukherjee,"director,editor,writer"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,Raoul Ruiz,"director,writer,producer"
...,...,...,...,...,...,...,...,...,...,...
86772,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6,Giancarlo Soldi,"director,writer,producer"
86773,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,Ahmet Faik Akinci,"director,writer"
86774,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
86775,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"


In [17]:
director_and_ratings_df["genres"] = director_and_ratings_df["genres"].str.split(pat = ",")
director_and_ratings_df = director_and_ratings_df.explode("genres")
director_and_ratings_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,director,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Harnam Singh Rawail,"director,writer,producer"
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime,7.0,77,Harnam Singh Rawail,"director,writer,producer"
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Drama,7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,Biography,7.2,43,Mani Kaul,"director,writer,actor"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,Drama,7.2,43,Mani Kaul,"director,writer,actor"
...,...,...,...,...,...,...,...,...,...,...
86773,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,Drama,8.7,136,Ahmet Faik Akinci,"director,writer"
86773,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,Family,8.7,136,Ahmet Faik Akinci,"director,writer"
86774,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
86775,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"


In [18]:
writers_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,writer,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Abrar Alvi,"writer,actor,director"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Gulzar,"music_department,writer,soundtrack"
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Mahasweta Devi,writer
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Anjana Rawail,"writer,costume_designer"
4,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,None,None
...,...,...,...,...,...,...,...,...,...,...
122379,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
122380,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Victoria Sloan Jordan,"writer,producer"
122381,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous"
122382,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Pep Puig,writer


In [20]:
writers_df["genres"] = writers_df["genres"].str.split(pat = ",")
writers_df = writers_df.explode("genres")
writers_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,writer,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Abrar Alvi,"writer,actor,director"
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime,7.0,77,Abrar Alvi,"writer,actor,director"
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Drama,7.0,77,Abrar Alvi,"writer,actor,director"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Gulzar,"music_department,writer,soundtrack"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime,7.0,77,Gulzar,"music_department,writer,soundtrack"
...,...,...,...,...,...,...,...,...,...,...
122379,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
122380,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Victoria Sloan Jordan,"writer,producer"
122381,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous"
122382,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Pep Puig,writer


In [32]:
writers_and_directors_df = writers_df.merge(director_and_ratings_df, how = "inner", on = "original_title")
writers_and_directors_df

,movie_id_x,primary_title_x,original_title,start_year_x,runtime_minutes_x,genres_x,avg_rating_x,num_votes_x,writer,primary_profession_x,movie_id_y,primary_title_y,start_year_y,runtime_minutes_y,genres_y,avg_rating_y,num_votes_y,director,primary_profession_y
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Abrar Alvi,"writer,actor,director",tt0063540,Sunghursh,2013,175.0,Action,7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Abrar Alvi,"writer,actor,director",tt0063540,Sunghursh,2013,175.0,Crime,7.0,77,Harnam Singh Rawail,"director,writer,producer"
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action,7.0,77,Abrar Alvi,"writer,actor,director",tt0063540,Sunghursh,2013,175.0,Drama,7.0,77,Harnam Singh Rawail,"director,writer,producer"
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime,7.0,77,Abrar Alvi,"writer,actor,director",tt0063540,Sunghursh,2013,175.0,Action,7.0,77,Harnam Singh Rawail,"director,writer,producer"
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime,7.0,77,Abrar Alvi,"writer,actor,director",tt0063540,Sunghursh,2013,175.0,Crime,7.0,77,Harnam Singh Rawail,"director,writer,producer"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827403,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,Drama,8.7,136,Ahmet Faik Akinci,"director,writer",tt9914286,Sokagin Çocuklari,2019,98.0,Family,8.7,136,Ahmet Faik Akinci,"director,writer"
827404,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,Family,8.7,136,Ahmet Faik Akinci,"director,writer",tt9914286,Sokagin Çocuklari,2019,98.0,Drama,8.7,136,Ahmet Faik Akinci,"director,writer"
827405,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,Family,8.7,136,Ahmet Faik Akinci,"director,writer",tt9914286,Sokagin Çocuklari,2019,98.0,Family,8.7,136,Ahmet Faik Akinci,"director,writer"
827406,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous",tt9914942,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"
